In [7]:
ls

DataScienceCaseStudy.Rmd    FightingMongoosesML/
DataScienceCaseStudy.html   data_description.txt
DataScienceCaseStudy_cache/ house_loaded.RData
DataScienceCaseStudy_files/ sample_submission.csv
ExploratoryMLProject        test.csv
ExploringData.ipynb         train.csv


In [89]:
import numpy as np
import pandas as pd
import re
pd.set_option('display.max_columns', 100)

In [9]:
RawTrainData = pd.read_csv('train.csv')
RawTestData = pd.read_csv('test.csv')

In [61]:
RawTrainData.head()
RawTestData.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [62]:
# Save the 'Id' column
train_ID = RawTrainData['Id']
test_ID = RawTestData['Id']

# Now drop the 'Id' colum since we can not use it as a feature to train our model.
RawTrainData.drop("Id", axis = 1, inplace = True)
RawTestData.drop("Id", axis = 1, inplace = True)

In [65]:
y_train = RawTrainData['SalePrice']
X_train = RawTrainData.drop('SalePrice', axis=1)
X_test = RawTestData.copy()

In [66]:
print(X_train.shape)
print(X_test.shape)

(1460, 79)
(1459, 79)


In [64]:
RawTrainData.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [67]:
all_data = pd.concat([X_train, X_test], ignore_index=True)
all_data.shape

(2919, 79)

In [23]:
sorted = RawTrainData.sort_values(by='SalePrice')

### Random thoughts

Feature Generation
- Look for month trends
- Maybe eliminate some outliers?
- Aggregate basemt + SF to get total
- Compute the price/sq ft, could also help with outliers
- Curious if 1st floor vs. second floor SF makes a big difference
- Time of sale probably matters a ton, the sample set spans the boom-bust range of the recession

In [24]:
byNeighborhood = sorted.groupby('Neighborhood').agg('count')
byNeighborhood.sort_values('SalePrice').iloc[:,30:50]

,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath
Neighborhood,,,,,,,,,,,,,,,,,,,,
Blueste,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
NPkVill,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9,9
Veenker,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11
BrDale,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16
MeadowV,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17
Blmngtn,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17,17
StoneBr,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25
SWISU,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25,25
ClearCr,27,27,27,28,27,28,28,28,28,28,28,28,28,28,28,28,28,28,28,28


In [45]:
sorted.iloc[1300:,:20]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt
1107,1108,60,RL,168.0,23257,Pave,NaN,IR3,HLS,AllPub,CulDSac,Gtl,Gilbert,Norm,Norm,1Fam,2Story,7,5,2006
199,200,20,RL,76.0,9591,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,Norm,Norm,1Fam,1Story,8,5,2004
1423,1424,80,RL,NaN,19690,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Edwards,Norm,Norm,1Fam,SLvl,6,7,1966
1318,1319,20,RL,NaN,14781,Pave,NaN,IR2,Lvl,AllPub,CulDSac,Gtl,CollgCr,Norm,Norm,1Fam,1Story,8,5,2001
720,721,120,RL,NaN,6563,Pave,NaN,IR1,Low,AllPub,CulDSac,Mod,StoneBr,Norm,Norm,1Fam,1Story,8,5,1985
1405,1406,120,RM,44.0,3842,Pave,NaN,IR1,HLS,AllPub,Inside,Mod,Crawfor,Norm,Norm,TwnhsE,1Story,8,5,2004
426,427,80,RL,NaN,12800,Pave,NaN,Reg,Low,AllPub,Inside,Mod,SawyerW,Norm,Norm,1Fam,SLvl,7,5,1989
600,601,60,RL,74.0,10927,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,Norm,Norm,1Fam,2Story,8,5,2005
753,754,60,RL,80.0,10240,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,Norm,Norm,1Fam,2Story,8,5,2005
261,262,60,RL,69.0,9588,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,8,5,2007


### Complete Basic Cleaning and Analysis


### 2 Data Loading and Prep

##### 2.1 Loading data from csv

In [68]:
RawTrainData.isnull().sum()

MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
BsmtQual           37
                 ... 
BedroomAbvGr        0
KitchenAbvGr        0
KitchenQual         0
TotRmsAbvGrd        0
Functional          0
Fireplaces          0
FireplaceQu       690
GarageType         81
GarageYrBlt        81
GarageFinish       81
GarageCars          0
GarageArea          0
GarageQual         81
GarageCond         81
PavedDrive

##### 2.2 Getting Factor levels from data description?


##### 2.3 Checking factor levels with data

In [126]:
for c in all_data.columns:
    if all_data[c].dtype == 'object':
        print(c, len(all_data[c].value_counts()))

MSZoning 5
Street 2
Alley 2
LotShape 4
LandContour 4
Utilities 2
LotConfig 5
LandSlope 3
Neighborhood 25
Condition1 9
Condition2 8
BldgType 5
HouseStyle 8
RoofStyle 6
RoofMatl 8
Exterior1st 15
Exterior2nd 16
MasVnrType 4
ExterQual 4
ExterCond 5
Foundation 6
BsmtQual 4
BsmtCond 4
BsmtExposure 4
BsmtFinType1 6
BsmtFinType2 6
Heating 6
HeatingQC 5
CentralAir 2
Electrical 5
KitchenQual 4
Functional 7
FireplaceQu 5
GarageType 6
GarageFinish 3
GarageQual 5
GarageCond 5
PavedDrive 3
PoolQC 3
Fence 4
MiscFeature 4
SaleType 9
SaleCondition 6


In [114]:
# get the number of each category for SaleType
for type in set(all_data.SaleType):
    print(type,sum(all_data.SaleType==type))
    
for type in set(all_data.SaleCondition):
    print(type,sum(all_data.SaleCondition==type))

nan 0
COD 87
WD 2525
Oth 7
CWD 12
Con 5
ConLw 8
New 239
ConLI 9
ConLD 26
Alloca 24
AdjLand 12
Family 46
Abnorml 190
Partial 245
Normal 2402


In [122]:
#Exploring just the Family sales
y_train[X_train.SaleCondition=='Family']
X_train[X_train.SaleCondition=='Family']
pd.concat([y_train[X_train.SaleCondition=='Family'],X_train[X_train.SaleCondition=='Family']],axis=1)

,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
137,171000,90,RL,82.0,11070,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,Duplex,1Story,7,5,1988,1989,Gable,CompShg,VinylSd,VinylSd,BrkFace,70.0,TA,TA,CBlock,TA,TA,No,Unf,0,Unf,0,1907,1907,GasA,Gd,Y,SBrkr,1959,0,0,1959,0,0,3,0,5,2,TA,9,Typ,0,NaN,2Types,1989.0,Unf,3,766,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,7,2006,WD,Family
154,125000,30,RM,84.0,11340,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,1Story,6,5,1923,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,1200,1200,GasA,TA,Y,FuseA,1200,0,0,1200,0,0,1,0,4,1,TA,7,Typ,0,NaN,Detchd,1923.0,Unf,1,312,Fa,Fa,Y,0,0,228,0,0,0,NaN,NaN,NaN,0,3,2006,WD,Family
217,107000,70,RM,57.0,9906,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,4,1925,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0,Unf,0,686,686,GasA,Fa,N,SBrkr,810,518,0,1328,0,0,1,0,3,1,TA,8,Typ,0,NaN,Detchd,1940.0,Unf,1,210,TA,TA,Y,0,172,60,0,0,0,NaN,NaN,NaN,0,9,2006,WD,Family
251,235000,120,RM,44.0,4750,Pave,NaN,IR1,HLS,AllPub,Inside,Mod,Crawfor,Norm,Norm,TwnhsE,1Story,8,5,2006,2007,Hip,CompShg,VinylSd,VinylSd,Stone,481.0,Gd,TA,PConc,Gd,TA,Gd,GLQ,1573,Unf,0,0,1573,GasA,Ex,Y,SBrkr,1625,0,0,1625,1,1,2,0,2,1,Gd,5,Typ,1,Gd,Attchd,2006.0,Fin,2,538,TA,TA,Y,123,0,0,0,153,0,NaN,NaN,NaN,0,12,2007,WD,Family
299,158500,20,RL,80.0,14000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,1Story,6,8,1950,2004,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,Gd,CBlock,TA,TA,No,Unf,0,Unf,0,1092,1092,GasA,Ex,Y,SBrkr,1152,0,0,1152,0,1,1,0,3,1,Gd,6,Typ,1,Gd,Attchd,1950.0,Unf,1,300,TA,TA,Y,0,36,0,0,0,0,NaN,GdPrv,NaN,0,8,2009,WD,Family
454,188000,90,RL,63.0,9297,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,Duplex,1Story,5,5,1976,1976,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1606,Unf,0,122,1728,GasA,TA,Y,SBrkr,1728,0,0,1728,2,0,2,0,4,2,TA,8,Typ,0,NaN,Detchd,1976.0,Unf,2,560,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,7,2006,WD,Family
628,135000,60,RL,70.0,11606,Pave,NaN,IR1,HLS,AllPub,Inside,Sev,NAmes,Norm,Norm,1Fam,2Story,5,5,1969,1969,Gable,CompShg,Plywood,Plywood,BrkFace,192.0,TA,TA,PConc,Gd,TA,Av,Rec,650,Unf,0,390,1040,GasA,TA,Y,SBrkr,1040,1040,0,2080,0,1,1,2,5,1,Fa,9,Typ,2,TA,Attchd,1969.0,Unf,2,504,TA,TA,Y,335,0,0,0,0,0,NaN,NaN,NaN,0,9,2007,WD,Family
632,82500,20,RL,85.0,11900,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,7,5,1977,1977,Hip,CompShg,Plywood,Plywood,BrkFace,209.0,TA,Gd,CBlock,TA,TA,No,ALQ,822,Unf,0,564,1386,GasA,TA,Y,SBrkr,1411,0,0,1411,0,0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1977.0,Fin,2,544,TA,TA,Y,192,0,0,0,0,0,NaN,NaN,NaN,0,4,2009,WD,Family
655,88000,160,RM,21.0,1680,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,BrDale,Norm,Norm,Twnhs,2Story,6,5,1971,1971,Gable,CompShg,HdBoard,ImStucc,BrkFace,381.0,TA,TA,CBlock,TA,TA,No,Unf,0,Unf,0,525,525,GasA,TA,Y,SBrkr,525,567,0,1092,0,0,1,1,3,1,TA,6,Typ,0,NaN,Detchd,1971.0,Unf,1,264,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,3,2010,WD,Family
734,108000,20,RL,NaN,8978,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,5,1968,1968,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,PConc,TA,TA,No,Unf,0,Unf,0,948,948,GasA,TA,Y,SBrkr,948,0,0,948,0,0

In [90]:
X_train[objectcols].head(100)

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,NaN,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,TA,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Mn,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,Wd Sdng,Wd Shng,None,TA,TA,BrkTil,TA,Gd,No,ALQ,Unf,GasA,Gd,Y,SBrkr,Gd,Typ,Gd,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
5,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,Gable,CompShg,VinylSd,VinylSd,None,TA,TA,Wood,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,NaN,Attchd,Unf,TA,TA,Y,NaN,MnPrv,Shed,WD,Normal
6,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,Stone,Gd,TA,PConc,Ex,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,Gd,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
7,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,Gable,CompShg,HdBoard,HdBoard,Stone,TA,TA,CBlock,Gd,TA,Mn,ALQ,BLQ,GasA,Ex,Y,SBrkr,TA,Typ,TA,Attchd,RFn,TA,TA,Y,NaN,NaN,Shed,WD,Normal
8,RM,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,Gable,CompShg,BrkFace,Wd Shng,None,TA,TA,BrkTil,TA,TA,No,Unf,Unf,GasA,Gd,Y,FuseF,TA,Min1,TA,Detchd,Unf,Fa,TA,Y,NaN,NaN,NaN,WD,Abnorml
9,RL,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,BrkTil,TA,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,TA,Attchd,RFn,Gd,TA,Y,NaN,NaN,NaN,WD,Normal


##### 2.4 Fixing Level Names


In [79]:
set(all_data.MSZoning)

{nan, 'RM', 'FV', 'C (all)', 'RL', 'RH'}

In [75]:
set(all_data.BldgType)

{'1Fam', '2fmCon', 'Duplex', 'Twnhs', 'TwnhsE'}

In [80]:
set(all_data.Neighborhood)

{'Blmngtn',
 'Blueste',
 'BrDale',
 'BrkSide',
 'ClearCr',
 'CollgCr',
 'Crawfor',
 'Edwards',
 'Gilbert',
 'IDOTRR',
 'MeadowV',
 'Mitchel',
 'NAmes',
 'NPkVill',
 'NWAmes',
 'NoRidge',
 'NridgHt',
 'OldTown',
 'SWISU',
 'Sawyer',
 'SawyerW',
 'Somerst',
 'StoneBr',
 'Timber',
 'Veenker'}

In [81]:
set(all_data.Exterior2nd)

{nan,
 'Stucco',
 'CmentBd',
 'Other',
 'Plywood',
 'Wd Shng',
 'MetalSd',
 'Brk Cmn',
 'Wd Sdng',
 'BrkFace',
 'ImStucc',
 'CBlock',
 'Stone',
 'VinylSd',
 'AsphShn',
 'AsbShng',
 'HdBoard'}

##### 2.5 Converting Column Data Types

In [54]:
RawTrainData.MSZoning = RawTrainData.MSZoning.astype('category')
RawTrainData.MSSubClass = RawTrainData.MSSubClass.astype('category')
RawTrainData.Street = RawTrainData.Street.astype('category')
RawTrainData.Alley = RawTrainData.Alley.astype('category')
RawTrainData.LotShape = RawTrainData.LotShape.astype('category')
RawTrainData.LandContour = RawTrainData.LandContour.astype('category')


In [ ]:
otherCategoricals = ['MSSubClass',]

In [124]:
ordinalFeatures = ['OverallQual','OverallCond','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','KitchenQual','Functional','FireplaceQu','GarageQual','GarageCond','GarageFinish''PoolQC','FenceQC','HeatingQC']

In [125]:
ordinalFeatures

['OverallQual',
 'OverallCond',
 'ExterQual',
 'ExterCond',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageQual',
 'GarageCond',
 'GarageFinishPoolQC',
 'FenceQC']

In [133]:
# Get a list of all the object types from the initial data
objectcols = []
for c in all_data.columns:
    if all_data[c].dtype == 'object':
        objectcols.append(c)

In [134]:
objectcols

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [111]:
# Convert all objects to categories?
for cols in objectcols:
    X_train[cols] = X_train[cols].astype('category')

MSZoning
Street
Alley
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Heating
HeatingQC
CentralAir
Electrical
KitchenQual
Functional
FireplaceQu
GarageType
GarageFinish
GarageQual
GarageCond
PavedDrive
PoolQC
Fence
MiscFeature
SaleType
SaleCondition


In [113]:
X_train.dtypes[1:45]#[45:]

MSZoning        category
LotFrontage      float64
LotArea            int64
Street          category
Alley           category
LotShape        category
LandContour     category
Utilities       category
LotConfig       category
LandSlope       category
Neighborhood    category
Condition1      category
Condition2      category
BldgType        category
HouseStyle      category
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle       category
RoofMatl        category
Exterior1st     category
Exterior2nd     category
MasVnrType      category
MasVnrArea       float64
ExterQual       category
ExterCond       category
Foundation      category
BsmtQual        category
BsmtCond        category
BsmtExposure    category
BsmtFinType1    category
BsmtFinSF1         int64
BsmtFinType2    category
BsmtFinSF2         int64
BsmtUnfSF          int64
TotalBsmtSF        int64
Heating         category
HeatingQC       category
CentralAir      category


##### 2.6 Saving Data?

### 3 Visualization

In [123]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

##### 3.1 SalePrice Histogram

##### 3.2 Plotting all features sorted by SalePrice

##### 3.3 Correlations between continuous variables

##### 3.4 Correlations between ordinal variables

##### 3.5 Correlations between nominal variables

##### 3.6 Correlations between ordinal variables and SalePrice

### 4 Pre-Processing

#### 4.1 Understanding missingness

#### 4.2 Missing Pattern of GarageYrBlt

#### 4.3 Imputing GarageYrBlt

#### 4.4 Transforming YearRemodAdd

#### 4.5 Zero and near Zero-Variance Predictors

#### 4.6 Imputing other Features using KNN

#### 4.7 Transforming SalePrice to log scale

#### 4.8 Saving data

### 5 Modeling

#### 5.1 Model Training and Parameter Tuning

#### 5.2 Data Splitting

#### 5.3 Resampling

### 6 Regressions

#### 6.1 Basic Linear Regression

#### 6.2 Linear Regression Variable Importance

#### 6.3 Performance Measures for Linear Regression

#### 7.1 Linear Regression with Elastic Net Regularization - Grid search with `train`

#### 7.2 Choosing the Parameters

#### 7.3 Regularization Feature Importance

#### 7.4 Analyzing the Regularized Regression Performance


#### 8.1 Tree-based Ensemble Models: Gradient Boosting Machines

#### 8.2 gbm Parameters

#### 8.3 Performance Measures for gbm